In [6]:
import numpy as np
import pandas as pd
from scipy import misc

In [7]:
path = "../NetDissect-Lite/dataset/broden1_224/"


In [8]:
index = pd.read_csv(path + "index.csv")
label = pd.read_csv(path + "label.csv")
category = pd.read_csv(path + "category.csv")

In [9]:
index.sample(5)

,image,split,ih,iw,sh,sw,color,object,part,material,scene,texture
14955,ade20k/ADE_val_00001771.jpg,train,224,224,112,112,ade20k/ADE_val_00001771_color.png,ade20k/ADE_val_00001771_object.png,ade20k/ADE_val_00001771_part_1.png,NaN,0.0,NaN
11372,pascal/2009_000962.jpg,train,224,224,112,112,pascal/2009_000962_color.png,pascal/2009_000962_object.png,pascal/2009_000962_part.png,NaN,NaN,NaN
27479,ade20k/ADE_train_00016961.jpg,train,224,224,112,112,ade20k/ADE_train_00016961_color.png,ade20k/ADE_train_00016961_object.png,ade20k/ADE_train_00016961_part_1.png;ade20k/AD...,NaN,59.0,NaN
17704,ade20k/ADE_train_00001875.jpg,train,224,224,112,112,ade20k/ADE_train_00001875_color.png,ade20k/ADE_train_00001875_object.png,NaN,NaN,772.0,NaN
7632,opensurfaces/22742.jpg,train,224,224,112,112,opensurfaces/22742_color.png,NaN,NaN,opensurfaces/22742_material.png,NaN,NaN


In [10]:
index_train = index[index["split"]== "train"]
index_train.head(10)

,image,split,ih,iw,sh,sw,color,object,part,material,scene,texture
0,opensurfaces/25605.jpg,train,224,224,112,112,opensurfaces/25605_color.png,NaN,NaN,opensurfaces/25605_material.png,NaN,NaN
1,opensurfaces/24183.jpg,train,224,224,112,112,opensurfaces/24183_color.png,NaN,NaN,opensurfaces/24183_material.png,NaN,NaN
2,opensurfaces/72705.jpg,train,224,224,112,112,opensurfaces/72705_color.png,NaN,NaN,opensurfaces/72705_material.png,NaN,NaN
3,opensurfaces/100509.jpg,train,224,224,112,112,opensurfaces/100509_color.png,NaN,NaN,opensurfaces/100509_material.png,NaN,NaN
4,ade20k/ADE_train_00003891.jpg,train,224,224,112,112,ade20k/ADE_train_00003891_color.png,ade20k/ADE_train_00003891_object.png,NaN,NaN,83.0,NaN
5,opensurfaces/97035.jpg,train,224,224,112,112,opensurfaces/97035_color.png,NaN,NaN,opensurfaces/97035_material.png,NaN,NaN
6,ade20k/ADE_train_00006107.jpg,train,224,224,112,112,ade20k/ADE_train_00006107_color.png,ade20k/ADE_train_00006107_object.png,ade20k/ADE_train_00006107_part_1.png,NaN,301.0,NaN
7,opensurfaces/20360.jpg,train,224,224,112,112,opensurfaces/20360_color.png,NaN,NaN,opensurfaces/20360_material.png,NaN,NaN
8,opensurfaces/6582.jpg,train,224,224,112,112,opensurfaces/6582_color.png,NaN,NaN,opensurfaces/6582_material.png,NaN,NaN
9,opensurfaces/97767.jpg,train,224,224,112,112,opensurfaces/97767_color.png,NaN,NaN,opensurfaces/97767_material.png,NaN,NaN


In [11]:
## Let's isolate just images with segmented objects 

objects_only = index[index["object"].notnull()][["image", "split", "ih", "iw", "sh", "sw", "color", "object"]]

## let's count the number of objects in each image
objects_only["object_count"] = objects_only["object"].apply(lambda x: len(x.split(";")))

objects_only.sort_values("object_count", ascending = True)

objects_only.head(10)

## So it seems like the images (despite the descriptions of the data) only have one object each.

,image,split,ih,iw,sh,sw,color,object,object_count
4,ade20k/ADE_train_00003891.jpg,train,224,224,112,112,ade20k/ADE_train_00003891_color.png,ade20k/ADE_train_00003891_object.png,1
6,ade20k/ADE_train_00006107.jpg,train,224,224,112,112,ade20k/ADE_train_00006107_color.png,ade20k/ADE_train_00006107_object.png,1
12,ade20k/ADE_train_00010928.jpg,train,224,224,112,112,ade20k/ADE_train_00010928_color.png,ade20k/ADE_train_00010928_object.png,1
17,pascal/2010_004760.jpg,train,224,224,112,112,pascal/2010_004760_color.png,pascal/2010_004760_object.png,1
18,ade20k/ADE_train_00018550.jpg,train,224,224,112,112,ade20k/ADE_train_00018550_color.png,ade20k/ADE_train_00018550_object.png,1
21,ade20k/ADE_train_00019640.jpg,train,224,224,112,112,ade20k/ADE_train_00019640_color.png,ade20k/ADE_train_00019640_object.png,1
23,ade20k/ADE_train_00017062.jpg,train,224,224,112,112,ade20k/ADE_train_00017062_color.png,ade20k/ADE_train_00017062_object.png,1
24,ade20k/ADE_train_00001215.jpg,train,224,224,112,112,ade20k/ADE_train_00001215_color.png,ade20k/ADE_train_00001215_object.png,1
26,ade20k/ADE_train_00011520.jpg,train,224,224,112,112,ade20k/ADE_train_00011520_color.png,ade20k/ADE_train_00011520_object.png,1
29,ade20k/ADE_train_00015917.jpg,train,224,224,112,112,ade20k/ADE_train_00015917_color.png,ade20k/ADE_train_00015917_object.png,1


In [12]:
label.shape
label.head()

,number,name,category,frequency,coverage,syns
0,1,black-c,color(62358),62358,11135.320474,NaN
1,2,grey-c,color(62310),62310,12712.843129,NaN
2,3,white-c,color(62054),62054,5778.038204,NaN
3,4,brown-c,color(61583),61583,14549.652454,NaN
4,5,green-c,color(61508),61508,5049.989735,NaN


In [37]:
category.head(6) # there are only 6 categories

,name,first,last,count,frequency
0,color,1,11,11,651776
1,object,12,1197,584,271047
2,part,12,1186,234,132414
3,material,13,891,32,54511
4,scene,59,1179,468,17746
5,texture,237,391,47,6611


In [31]:
## We want to produce three sets of data of different sizes

In [63]:
image = misc.imread(path + "images/" + "ade20k/ADE_train_00003891_color.png")
print (image)

[[[ 4  0  0]
  [ 4  0  0]
  [ 4  0  0]
  ..., 
  [ 6  0  0]
  [ 6  0  0]
  [ 0  0  0]]

 [[ 4  0  0]
  [ 4  0  0]
  [ 4  0  0]
  ..., 
  [ 4  0  0]
  [ 4  0  0]
  [ 0  0  0]]

 [[ 4  0  0]
  [ 4  0  0]
  [ 4  0  0]
  ..., 
  [ 4  0  0]
  [ 4  0  0]
  [ 0  0  0]]

 ..., 
 [[ 4  0  0]
  [11  0  0]
  [10  0  0]
  ..., 
  [11  0  0]
  [11  0  0]
  [ 0  0  0]]

 [[11  0  0]
  [10  0  0]
  [11  0  0]
  ..., 
  [ 4  0  0]
  [ 4  0  0]
  [ 0  0  0]]

 [[11  0  0]
  [10  0  0]
  [10  0  0]
  ..., 
  [ 4  0  0]
  [ 4  0  0]
  [ 0  0  0]]]


In [ ]:
## 